In [1]:
import json
import random
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Upload dei file
La variabile **corpus** è una lista delle:
- frasi contenute negli articoli 
- domande delle FAQ

In [2]:
corpus = list()

In [3]:
with open ('articoli.txt', 'r', encoding = 'utf-8') as file:
    corpus = json.loads(file.read())
    
# eliminazione di quelle frasi che sono delle domande
# (per evitare che il chatbot risponda con delle domande)
for sentence in corpus:
    if (sentence[len(sentence)-1] == '?'):
        corpus.remove(sentence)

**q_a** dizionario che ha per chiave le domande delle FAQ e per valore una tupla contenente la risposta alla FAQ e il link della pagina web in cui le FAQ sono state estratte

In [4]:
with open('q&a.json', 'r') as data_structure:
    q_a = json.load(data_structure)
    # appendiamo le chiavi del dizionario (le domande delle FAQ)
    for k in q_a.keys():
        corpus.append(k)

### Costruzione del modello
Tf-idf converte una collezione di testi in una matrice di valori tf-idf.

Parametri di TfidfVectorizer:
- ngram_range = range minimo e massimo di possibili valori che n può assumere per l'estrazione degli n-grammi
- min_df / maz_df = quando costruisce il vocabolario, il modello ignora i termini la cui document frequency è minore / maggiore del numero indicato

**fit_transform** = impara il vocabolario e restituisce la document-term matrix (NB: nel nostro caso i documenti sono le frasi del nostro corpus)

In [5]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_df = 1000)
# applichiamo il modello
matrix_tfidf = tfidf.fit_transform(corpus)

**cosine_similarity** = funzione di scikit-learn che calcola la cosine similarity tra X e Y

In [6]:
# funzione che calcola e restituisce la cosine similarity tra
# un vettore e la document-term matrix
def compute_similarity (model, matrix, query):
    vec_query = model.transform([query])
    # flatten () rende la similarity matrix unidimensionale
    cos_sim = cosine_similarity(vec_query, matrix).flatten()
    
    return cos_sim

### Impostazione chatbot

In [7]:
# funzione che restituisce un saluto come risposta a un saluto dell'utente
def greeting_response(text):
    text = text.lower()
    
    # lista dei possibili saluti che il bot può dare
    bot_greetings = ['ciao', 'ehi']
    # lista dei saluti che il bot riconosce
    user_greetings = ['ciao']
    
    # se una parola dell'utente contiene una parola in user_greetings
    for word in text.split():
        if word in user_greetings:
            # restituisci una parola presa a caso da bot_greetings
            return random.choice(bot_greetings)

In [8]:
# funzione per riordinare una lista in ordine decrescente
def index_sort(list_var):
    length = len(list_var)
    # creo una lista di indici per la lunghezza 
    # della lista passata. Questa lista verrà 
    # e riordinata e restituita  alla fine
    list_index = list(range(0, length))
    x = list_var
    for i in range(length):
        for j in range(length):
            if x[list_index[i]] > x[list_index[j]]:
                # scambio gli elementi
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
                
    return list_index

In [9]:
def bot_response(user_input, model, matrix, q_a):
    bot_response = ''
    # flag per vedere se esiste una risposta per la query dell'utente
    response_flag = 0
    
    similarity_scores_list = compute_similarity(model, matrix, user_input)
    # index = lista di indici riodinati a seconda del similarity score
    index = index_sort(similarity_scores_list)
    
    # contatore per attaccare solo la prima risposta 
    # che abbia similarity score > 0.0
    j = 0
    for i in range(len(index)):
        if j >= 1:
            break
        # se il similarity score è maggiore di 0, 
        # significa una risposta esiste
        if similarity_scores_list[index[i]] > 0.0:
            # se la frase più simile che ha trovato è una domanda delle FAQ
            # restituisco la risposta
            if corpus[index[i]] in q_a.keys():
                bot_response = bot_response + q_a[corpus[index[i]]][0]
                response_flag = 1 # abbiamo trovato la risposta
                j = j + 1 # aggiorno il contatore per ogni risposta trovata
            # altrimenti, prende una frase degli articoli
            else:
                bot_response = bot_response + corpus[index[i]]
                response_flag = 1 # abbiamo trovato la risposta
                j = j + 1 # aggiorno il contatore per ogni risposta trovata

    # se non abbiamo trovato una risposta
    if response_flag == 0:
        bot_response = bot_response + 'Scusa, non sono in grado di risponderti.'

    return bot_response

### Esempio di conversazione (try it yourself!)

In [10]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

exit_list = ['esci']
while(True):
    print()
    print('User:')
    user_input = input()
    if user_input.lower() in exit_list:
        print('Co-bot: Ciao, a presto!')
        break
    else:
        if greeting_response(user_input) != None:
            print('Co-bot:')
            print(greeting_response(user_input))
        else:
            print('Co-bot: ... ')
            print(bot_response(user_input, tfidf, matrix_tfidf, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:


KeyboardInterrupt: Interrupted by user

### Testing
Il nostro test set è una lista di 42 domande che sono state raccolte con le modalità descritte nel report.

In [11]:
test_set = ["Cosa fare nel caso in cui si manifestano i sintomi?", 
            "Posso viaggiare se ho il coronavirus?", 
            "Le donne in gravidanza possono trasmettere la malattia al figlio?", 
            "Quanto tempo resta nel corpo il coronavirus?", 
            "Come funziona il contact tracing?", 
            "Quando finirà la pandemia?", 
            "Il vaccino ha effetti collaterali?", 
            "Cosa sappiamo della nuova variante inglese?", 
            "Quanti tipi di vaccino esistono?", 
            "Fino a quando saranno chiuse le attività commerciali?", 
            "Come distinguiamo un'influenza stagionale da un'influenza covid?", 
            "Chi si è vaccinato può trasmette la malattia?", 
            "Quali sono i colori che possono assumere le regioni?", 
            "Come fanno le persone infette a trasmettere il virus?", 
            "Se vengo dall'estero devo fare il tampone?", 
            "Cos’è l'indice Rt?", 
            "Posso ammalarmi due volte di covid?", 
            "Quanto tempo dura l'isolamento fiduciario?", 
            "È possibile fare sport di contatto?", 
            "Come faccio a capire se sono positiva ma asintomatica?", 
            "Sono stato a contatto con un infetto: cosa devo fare?", 
            "Fino a quando resteranno chiuse le piscine e le palestre?", 
            "Dopo il vaccino, devo comunque continuare a usare la mascherina?", 
            "Che cosa devo fare se voglio andare all'estero?", 
            "Lo smog/inquinamento influisce sulla diffusione del coronavirus/covid?", 
            "Qual è il periodo di copertura del vaccino?", 
            "Se sono all’estero, cosa devo fare per tornare in Italia?", 
            "Come proteggere le persone più anziane dal virus?", 
            "Il coronavirus è pericoloso per gli animali domestici?", 
            "La mascherina è obbligatoria all’aperto?", 
            "Fino a quando sarà adottata la didattica a distanza per le scuole superiori?", 
            "Qual è l'incidenza del covid nei bambini?", 
            "I guanti servono a proteggere dal contagio?", 
            "Si può viaggiare fuori comune in area gialla?", 
            "Quanto sono importanti il distanziamento e la chiusura delle attività nella trasmissione del covid?", 
            "Posso fare attività sportiva se sono positivo asintomatico?", 
            "Per i bambini è obbligatorio l’uso della mascherina?", 
            "Come agiscono i linfociti T nelle infezioni da covid?", 
            "Posso visitare i miei nonni?", 
            "È possibile andare a pesca?", 
            "Quali sono i sintomi del coronavirus?", 
            "Il virus può rimanere sulle superfici?"]

In [13]:
print('Co-bot:')
print('Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus')
print()

for sentence in test_set:
    user_input = sentence
    
    print()
    print('User:')
    print (user_input)

    print('Co-bot: ... ')
    print(bot_response(user_input, tfidf, matrix_tfidf, q_a))

Co-bot:
Benvenuto! Risponderò a tutti i tuoi dubbi riguardo il coronavirus


User:
Cosa fare nel caso in cui si manifestano i sintomi?
Co-bot: ... 
Nel caso in cui compaiano sintomi è necessario l’utilizzo di mascherine certificate come dispositivi medici.

User:
Posso viaggiare se ho il coronavirus?
Co-bot: ... 
Ecco le misure da adottare: Portare sempre con sé dispositivi di protezione delle vie respiratorie e indossarli nei luoghi al chiuso e in tutti i luoghi all'aperto quando non possa essere garantita in modo continuativo la condizione di isolamento rispetto a persone non conviventi. È fortemente raccomandato l'uso dei dispositivi di protezione delle vie respiratorie anche all'interno delle abitazioni private in presenza di persone non conviventi. ​Sono esonerati dall’obbligo di indossare la mascherina: i soggetti che stanno svolgendo attività sportiva i bambini di età inferiore ai sei anni i soggetti con patologie o disabilità incompatibili con l'uso della mascherina e chi inter

L’ AIFA(https://www.aifa.gov.it/) (Agenzia Italiana del Farmaco), oltre alle attività di farmacovigilanza normalmente previste per farmaci e vaccini (basate sulle segnalazioni spontanee e sulle reti di farmacovigilanza già presenti), promuoverà l’avvio di alcuni studi indipendenti post-autorizzativi sui vaccini COVID-19. Le attività di vigilanza riguarderanno sia la raccolta e valutazione delle segnalazioni spontanee di sospetta reazione avversa (farmacovigilanza passiva) che azioni proattive, attraverso studi/progetti di farmaco-epidemiologia (farmacovigilanza attiva). L’AIFA si è dotata di un Comitato scientifico(https://www.aifa.gov.it/-/l-aifa-istituisce-il-comitato-scientifico-per-la-sorveglianza-dei-vaccini-covid-19) , che, per tutto il periodo della campagna vaccinale, avrà la funzione di supportare l’Agenzia e i responsabili scientifici dei singoli studi nella fase di impostazione delle attività, nell'analisi complessiva dei dati che saranno raccolti e nell’individuazione di po

Le evidenze scientifiche al momento disponibili indicano che il tempo di sopravvivenza del virus sulle superfici vari in relazione al tipo di superficie considerata. Il recente rapporto dell'Istituto superiore di sanità(http://www.iss.it/documents/20126/0/Rappporto+ISS+COVID-19+n.+25_2020.pdf/90decdd1-7c29-29e4-6663-b992e1773c98?t=1589836083759) riguardo le raccomandazioni sulla sanificazione di strutture non sanitarie nell’attuale emergenza COVID-19 (superfici, ambienti interni e abbigliamento) e la circolare 22 maggio 2020(http://www.trovanorme.salute.gov.it/norme/renderNormsanPdf?anno=2020&codLeg=74133&parte=1%20&serie=null) del ministero della Salute riportano i tempi di rilevazione di particelle virali sulle superfici più comuni, variabili da alcune ore (come ad es. sulla carta) fino a diversi giorni (come sulla plastica e l’acciaio inossidabile). Tuttavia, bisogna considerare che i dati finora disponibili, essendo generati da condizioni sperimentali, devono essere interpretati co

Ilprimoasse è relativo alla centralizzazione e gratuità del vaccino.

User:
Se sono all’estero, cosa devo fare per tornare in Italia?
Co-bot: ... 
La Circolare ministeriale 12 ottobre 2020(https://www.trovanorme.salute.gov.it/norme/renderNormsanPdf?anno=2020&codLeg=76613&parte=1%20&serie=null) stabilisce i criteri per porre fine all'isolamento dei casi confermati COVID-19. Caso COVID-19 positivo senza sintomi : può rientrare in comunità dopo un periodo di isolamento di almeno 10 giorni dalla comparsa della positività, al termine del quale risulti eseguito un test molecolare con risultato negativo (10 giorni + test). Caso COVID-19 positivo con sintomi : può rientrare in comunità dopo un periodo di isolamento di almeno 10 giorni dalla comparsa dei sintomi (non considerando anosmia e ageusia/disgeusia che possono avere prolungata persistenza nel tempo) accompagnato da un test molecolare con riscontro negativo eseguito dopo almeno 3 giorni senza sintomi (10 giorni, di cui almeno 3 giorni s

L’uso della mascherina non sarà obbligatorio durante lo svolgimento dell’attività sportiva.

User:
Come agiscono i linfociti T nelle infezioni da covid?
Co-bot: ... 
In caso di patologie diverse da COVID-19, con test diagnostico negativo, il soggetto rimarrà a casa fino a guarigione clinica seguendo le indicazioni del PLS/MMG che redigerà una attestazione che l’alunno/operatore scolastico può rientrare scuola poiché è stato seguito il percorso diagnostico-terapeutico e di prevenzione per COVID-19, come disposto da documenti nazionali e regionali.

User:
Posso visitare i miei nonni?
Co-bot: ... 
Ecco le misure da adottare: Portare sempre con sé dispositivi di protezione delle vie respiratorie e indossarli nei luoghi al chiuso e in tutti i luoghi all'aperto quando non possa essere garantita in modo continuativo la condizione di isolamento rispetto a persone non conviventi. È fortemente raccomandato l'uso dei dispositivi di protezione delle vie respiratorie anche all'interno delle abitazi